In [4]:
##讀取檔案 取出大表格
from bs4 import BeautifulSoup as bs
def get_response_element (file_name):
    f = open(file_name, 'r')
    response_text = f.read()
    f.close()
    soup = bs(response_text)
    tender_table = soup.select('.table_block.tender_table')[0]    #實際上的CLASS物件名稱為table_block tender_table  程式中用空白來代表.
    tr = tender_table.select('tr')
    return tr

In [5]:
##轉換時間格式 
from datetime import datetime
import os, sys, re
def date_conversion(element):
    m = re.match(r"(\d+/\d+/\d+)",element)
    if m is not None:
        t = m.group(1).split('/',1)
        if t[0] != '':
            return datetime.strptime(str(int(t[0]) + 1911) + '/' + (''.join(t[1:]).split()[0] ),"%Y/%m/%d")
        else:
            return None

In [6]:
##取出並轉換金錢格式
def money_conversion(element):
    m = re.match(r"\$?-?([0-9,]+)[元]?",element)
    return int(''.join(m.group(1).split(',')))

In [7]:
##去掉空白
def remove_space(element):
    return ''.join(element.split())

In [8]:
#function map 上述三個方法 只要對應到 就會傳讀入的參數來使用方法
name_map = {"機關代碼":"entity_code", "機關名稱":"procuring_entity","標案案號":"job_number","招標方式":"procurement_type",\
            "決標方式":"tender_awarding_type","標案名稱":"subject_of_procurement", "決標資料類別":"attr_of_tender_awarding", \
            "標的分類":"attr_of_procurement", "預算金額":"budget_value", "開標時間":"opening_date","決標公告日期":"tender_awarding_announce_date",\
            "歸屬計畫類別":"project_type","總決標金額":"total_tender_awarding_value","底價金額":"floor_price_value",\
            "決標日期":"tender_awarding_date", "pkAtmMain":"pkAtmMain"} 

tender_awards_map = {"機關代碼":remove_space, "機關名稱":remove_space, "標案案號":remove_space, \
                     "招標方式":remove_space,"決標方式":remove_space,"標案名稱":remove_space, "決標資料類別":remove_space, \
                     "標的分類":remove_space, "預算金額":money_conversion, "開標時間":date_conversion,"歸屬計畫類別":remove_space, \
                        "總決標金額":money_conversion,"底價金額":money_conversion,"決標日期":date_conversion, "決標公告日期":date_conversion}    

In [9]:
##整理整理表格進入字典並轉換為英文名稱 #用上面三個方法清理後塞入資料
def get_award_info_dic (element):
    returned_dic = {}
    for row in element:
        th = row.select('th')
        if len(th) > 0:
            th_name = th[0].text.encode('utf-8').strip()
            if ('award_table_tr_1' or \
                'award_table_tr_2' or \
                'award_table_tr_6' in row.get('class')) \
                 and (th_name in tender_awards_map):
                returned_dic[name_map[th_name]] = \
                    tender_awards_map[th_name](row.select('td')[0].text.strip())
    return returned_dic

In [ ]:
tr_ele =  get_response_element('gov/51642102_N10409.txt')
return_dic =  get_award_info_dic(tr_ele)

In [ ]:
#印出轉換結果
for f in return_dic:
    print f+' : ' ,return_dic[f]

In [ ]:
##資料庫階段
import sqlite3
db = sqlite3.connect('tender.sqlite')
cur = db.cursor()



In [ ]:
#建立資料庫表格FORMAT
cur = db.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS Tender_awards(
 id INTEGER PRIMARY KEY,
 pkAtmMain TEXT,
 procuring_entity TEXT,
 entity_code TEXT,
 attr_of_procurement TEXT,
 opening_date DATETIME,
 procurement_type TEXT,
 tender_awarding_type TEXT,
 project_type TEXT,
 subject_of_procurement TEXT,
 job_number TEXT,
 budget_value BIGINTEGER,
 attr_of_tender_awarding TEXT,
 floor_price_value BIGINTEGER,
 tender_awarding_announce_date DATETIME,
 tender_awarding_date DATETIME,
 total_tender_awarding_value BIGINTEGER
 )''')
db.close()

In [ ]:
##INSERT DATA 到資料庫的命令打法格式測試
placeholders =  ', '.join(return_dic.keys())
values =  ', '.join('?' * len(return_dic))

skeleton = 'insert into Tender_awards({}) values({});'
print skeleton.format(placeholders, values)

In [ ]:
## 開始IMPORT資料
# -*- coding: utf-8 -*-
import sqlite3
db = sqlite3.connect('tender.sqlite')
cur = db.cursor()
tr_ele =  get_response_element('gov/51642102_N10409.txt')
return_dic =  get_award_info_dic(tr_ele)
placeholders =  ', '.join(return_dic.keys())
values =  ', '.join('?' * len(return_dic))

skeleton = 'insert into Tender_awards({}) values({});'

cur.execute(skeleton.format(placeholders, values), return_dic.values())
db.commit()
db.close()

In [10]:
def insert_award_info (cur, data_dic, filename):
    file_param = filename.split(".txt")[0].split("_")
    pkAtmMain = file_param[0]
    data_dic['pkAtmMain'] = pkAtmMain
    columns = ', '.join(data_dic.keys())
    placeholders = ', '.join('?' * len(data_dic))
    sql = 'INSERT INTO Tender_awards ({}) VALUES ({})'.format(columns, placeholders)
    cur.execute(sql, data_dic.values())

In [ ]:
#印出資料夾內的文件
db = sqlite3.connect('tender.sqlite')
cur = db.cursor()
path = "gov/"
dirs = os.listdir(path) 
print dirs

In [11]:
#彙整上面的DEF式
import sqlite3
import os
#正式用for import資料
db = sqlite3.connect('tender.sqlite')
cur = db.cursor()
path = "gov/"
dirs = os.listdir(path) 
for filename in dirs:
    ele = get_response_element(path + filename)
    award_info_dic = get_award_info_dic(ele)    
    insert_award_info(cur, award_info_dic, filename)
db.commit()
db.close()

c:\python27\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))
